# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global data
    x_axis = []
    for i in range(data['sma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global data
    x_axis = []
    for i in range(data['lma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(row['tick_avg'])    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(row['tick_avg'])
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(row['tick_avg'])    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(row['tick_avg'])
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

## File paths

In [5]:
year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [6]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 731 ms


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


## Data manipulation

In [7]:
%%time
data = {}
data['number_of_ticks'] = 20
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████| 50000/50000 [00:11<00:00, 4218.79it/s]


Records : 50000
Wall time: 12.3 s


,tick_avg,spread_avg,tick_sd
0,1.200982,0.000382,0.000245
1,1.200657,0.000704,0.000092
2,1.200720,0.000756,0.000032
3,1.200684,0.000365,0.000109
4,1.200634,0.000248,0.000075


In [8]:
%%time

data['pip_diff'] = 0.00001
data['rsi_window'] = 14
data['rs_max'] = 1e6

data['sma_len'] = 20
data['lma_len'] = 50

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 47.9 ms


In [9]:
%%time

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    # Build sema ---------------------------------------------
    if len(data['ssma_list']) < data['sma_len']:
        before_sma()          
    elif len(data['ssma_list']) == data['sma_len']:
        after_sma() 
        df['sema'][i] = data['sema']
        df['sema_diff'][i]= data['sema_diff']
        
    # Build lema ---------------------------------------------
    if len(data['lsma_list']) < data['lma_len']:
        before_lma()     
    elif len(data['lsma_list']) == data['lma_len']:
        after_lma() 
        df['lema'][i] = data['lema']
        df['lema_diff'][i]= data['lema_diff']    
        
df = df.dropna() 

100%|███████████████████████████████████| 50000/50000 [02:21<00:00, 352.79it/s]

Wall time: 2min 21s


In [10]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Slopes -----------------------------
#df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope_s)
#df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope_l)

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 49948
Wall time: 40.6 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,ssma_diff,lsma,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,direction
49943,1.229165,0.000038,0.000023,1.22893,1.22892,2.82931e-05,8.56681e-06,0.000060,0.000040,0.000023,...,0.000012,1.228930,-4.150000e-06,-0.000005,1.229165,1.228743,0.0,-0.000422,1.03974e-05,increase
49944,1.229189,0.000037,0.000027,1.22896,1.22893,2.73829e-05,9.58655e-06,0.000024,0.000033,0.000023,...,0.000010,1.228927,-2.735000e-06,0.000008,1.229189,1.228743,0.0,-0.000445,2.81937e-05,increase
49945,1.229230,0.000033,0.000027,1.22899,1.22894,2.82327e-05,1.18935e-05,0.000041,0.000036,0.000021,...,0.000009,1.228928,7.050000e-07,0.000016,1.229230,1.228743,0.0,-0.000487,4.45329e-05,increase
49946,1.229292,0.000036,0.000019,1.22902,1.22896,3.30755e-05,1.41244e-05,0.000062,0.000040,0.000006,...,0.000014,1.228929,1.540000e-06,0.000028,1.229292,1.228743,0.0,-0.000549,6.3484e-05,increase
49947,1.229321,0.000043,0.000016,1.22906,1.22897,3.34149e-05,1.57488e-05,0.000029,0.000042,0.000005,...,0.000017,1.228934,4.965000e-06,0.000040,1.229321,1.228743,0.0,-0.000578,8.11501e-05,same


## Write data to csv

In [11]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 1.6 s


## Print Report

In [12]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       20861   41.765436
increase   14684   29.398575
decrease   14403   28.835989


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,ssma_diff,lsma,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,ema_diff,direction
0,1.201499,0.000148,0.000020,1.20146,1.20122,7.33704e-06,1.63754e-05,0.000005,0.000011,0.000012,...,1.128750e-05,1.201067,0.000016,0.000380,1.201534,1.201319,0.000035,-0.000180,0.000244872,same
1,1.201461,0.000141,0.000026,1.20147,1.20123,1.3611e-06,1.42081e-05,-0.000038,0.000010,0.000015,...,5.875000e-06,1.201082,0.000016,0.000370,1.201534,1.201319,0.000074,-0.000142,0.000232025,decrease
2,1.201332,0.000210,0.000022,1.20145,1.20124,-1.27295e-05,8.14962e-06,-0.000128,0.000010,0.000022,...,-7.500000e-08,1.201096,0.000014,0.000356,1.201534,1.201319,0.000202,-0.000014,0.000211146,decrease
3,1.201305,0.000126,0.000020,1.20144,1.20125,-1.46361e-05,4.60951e-06,-0.000028,0.000009,0.000024,...,-1.675000e-06,1.201103,0.000007,0.000347,1.201534,1.201305,0.000230,0.000000,0.0001919,decrease
4,1.201228,0.000100,0.000056,1.20142,1.20125,-2.13796e-05,8.80187e-08,-0.000076,0.000009,0.000029,...,-4.537500e-06,1.201106,0.000003,0.000340,1.201534,1.201228,0.000306,0.000000,0.000170432,decrease
